In [1]:
import os
import sys

os.environ["PYSPARK_PYTHON"]="C:\Spark\spark-3.3.2-bin-hadoop2\python"
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-18.0.2.1"
os.environ["SPARK_HOME"] = "C:\Spark\spark-3.3.2-bin-hadoop2"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.9.5-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")


MAX_MEMORY = "14G"
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("eda") \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()    
spark

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns 
from handyspark import * 

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [3]:
cluster_data = spark.read.option("header","true").csv("../data/EDA/clustering_data_all.csv")
cluster_data = cluster_data.drop("_c0")
cluster_data.printSchema()

print(cluster_data.count())

root
 |-- subject_id: string (nullable = true)
 |-- hadm_id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- bg2_lactate: string (nullable = true)
 |-- db_wbc: string (nullable = true)
 |-- basophils_abs: string (nullable = true)
 |-- eosinophils_abs: string (nullable = true)
 |-- lymphocytes_abs: string (nullable = true)
 |-- monocytes_abs: string (nullable = true)
 |-- neutrophils_abs: string (nullable = true)
 |-- basophils: string (nullable = true)
 |-- eosinophils: string (nullable = true)
 |-- lymphocytes: string (nullable = true)
 |-- monocytes: string (nullable = true)
 |-- neutrophils: string (nullable = true)
 |-- albumin: string (nullable = true)
 |-- aniongap: string (nullable = true)
 |-- bicarbonate: string (nullable = true)
 |-- bun: string (nullable = true)
 |-- calcium: string (nullable = true)
 |-- chloride: string (nullable = true)
 |-- creatinine: string (nullable = true)
 |-- glucose: string (nullable = true)
 |-- sodium: string (nullable = true)
 

In [4]:
predict_data = spark.read.option("header","true").csv("../data/EDA/prediction_value_all.csv")
predict_data = predict_data.drop("_c0")
predict_data.printSchema()

print(predict_data.count())

root
 |-- subject_id: string (nullable = true)
 |-- charlson_comorbidity_index: string (nullable = true)

138274


In [5]:
model_data = cluster_data.join(predict_data, cluster_data.subject_id == predict_data.subject_id, "inner")
print(model_data.count())

138274


In [6]:
model_data.printSchema()
model_data = model_data.drop("subject_id")
model_data.printSchema()

root
 |-- subject_id: string (nullable = true)
 |-- hadm_id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- bg2_lactate: string (nullable = true)
 |-- db_wbc: string (nullable = true)
 |-- basophils_abs: string (nullable = true)
 |-- eosinophils_abs: string (nullable = true)
 |-- lymphocytes_abs: string (nullable = true)
 |-- monocytes_abs: string (nullable = true)
 |-- neutrophils_abs: string (nullable = true)
 |-- basophils: string (nullable = true)
 |-- eosinophils: string (nullable = true)
 |-- lymphocytes: string (nullable = true)
 |-- monocytes: string (nullable = true)
 |-- neutrophils: string (nullable = true)
 |-- albumin: string (nullable = true)
 |-- aniongap: string (nullable = true)
 |-- bicarbonate: string (nullable = true)
 |-- bun: string (nullable = true)
 |-- calcium: string (nullable = true)
 |-- chloride: string (nullable = true)
 |-- creatinine: string (nullable = true)
 |-- glucose: string (nullable = true)
 |-- sodium: string (nullable = true)
 

In [7]:
# converting all number columns to double of precision 2 except for gender
import pyspark.sql.functions as F

cols = model_data.columns
cols.remove("gender")
model_data = model_data.select(*(F.round(F.col(c).cast("double"), 2).alias(c) for c in cols), "gender")
print(model_data.columns)
print(model_data.printSchema())
model_data.describe().toPandas()

['hadm_id', 'age', 'bg2_lactate', 'db_wbc', 'basophils_abs', 'eosinophils_abs', 'lymphocytes_abs', 'monocytes_abs', 'neutrophils_abs', 'basophils', 'eosinophils', 'lymphocytes', 'monocytes', 'neutrophils', 'albumin', 'aniongap', 'bicarbonate', 'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium', 'potassium', 'inr', 'pt', 'ptt', 'hematocrit', 'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet', 'rbc', 'rdw', 'wbc', 'scr_min', 'ckd', 'mdrd_est', 'scr_baseline', 'alt', 'alp', 'ast', 'bilirubin_total', 'charlson_comorbidity_index', 'gender']
root
 |-- hadm_id: double (nullable = true)
 |-- age: double (nullable = true)
 |-- bg2_lactate: double (nullable = true)
 |-- db_wbc: double (nullable = true)
 |-- basophils_abs: double (nullable = true)
 |-- eosinophils_abs: double (nullable = true)
 |-- lymphocytes_abs: double (nullable = true)
 |-- monocytes_abs: double (nullable = true)
 |-- neutrophils_abs: double (nullable = true)
 |-- basophils: double (nullable = true)
 |-- eosinophils:

,summary,hadm_id,age,bg2_lactate,db_wbc,basophils_abs,eosinophils_abs,lymphocytes_abs,monocytes_abs,neutrophils_abs,basophils,eosinophils,lymphocytes,monocytes,neutrophils,albumin,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,inr,pt,ptt,hematocrit,hemoglobin,mch,mchc,mcv,platelet,rbc,rdw,wbc,scr_min,ckd,mdrd_est,scr_baseline,alt,alp,ast,bilirubin_total,charlson_comorbidity_index,gender
0,count,138274,138274,58507,138270,80650,80657,80697,80650,80650,80650,80650,80697,80650,80650,63190,137068,137076,137227,126462,137105,137333,137056,137117,137123,103050,103047,101310,138274,138267,138264,138264,138265,138250,138266,138260,138270,137333,138274,138274,138271,79076,77423,79143,77257,138274,138274
1,mean,2.499391976303571E7,60.41605681473008,2.4679610986719545,11.623239965285308,0.061629758214501465,0.24784259270739148,2.3819698377882728,0.8265340359578421,9.465760570365767,0.4976875387476724,2.209233725976434,20.90972279019048,7.075853688778672,73.45839305641655,3.572750435195445,15.428247293314268,27.374315708074356,22.388545985848268,9.028962850500559,105.4525582582692,1.2342083111852225,149.74754844735,140.81610595330994,4.4265700867104725,1.4789007278020385,16.113068114549687,44.41841348336795,36.97107843846277,12.286014739598034,30.378053578661135,33.817583029566606,91.22566376161717,267.9683978300181,4.114893900163462,14.482467090988019,11.623223331163661,1.2342083111852225,0.10547897652487091,0.9571267917324091,1.096264147941366,110.31803581364763,121.09907908502642,148.9965000063177,1.4387537698849087,3.809985969885879,None
2,stddev,2885894.2800949407,18.91649106687153,2.230548678025936,35.08488045271723,0.5571837812872238,2.283558660151683,12.769101539071999,5.100963142445107,39.05418545631852,0.6229514653166558,2.990127534424857,13.845744231847279,4.6529956057522615,14.19823015501495,0.6423339008026292,3.627642863436428,3.6565743941473072,18.623515339532446,0.6975102082730493,4.822480060193525,1.3241542927397936,94.43342586880074,3.82539593651406,0.6678770470878982,0.9786221367404813,9.618174047322565,32.190308572674475,5.2813915699748275,1.8823381165976447,2.6126542361123675,1.493275670553355,6.93433482951141,125.1621111109577,0.6391813389793894,2.2026447421808473,35.08488183144825,1.3241542927397936,0.3071707088978358,0.13848245260328665,1.1563098509100584,550.6007050932291,140.08043349062447,845.6609116388131,3.5840468148517597,3.0922543898740877,None
3,min,2.0000019E7,18.0,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,3.0,4.0,1.0,2.6,77.0,0.1,26.0,97.0,1.6,0.6,7.1,16.0,13.4,3.4,12.7,22.4,50.0,9.0,1.35,10.6,0.2,0.1,0.0,0.76,0.1,1.0,6.0,1.0,0.1,0.0,F
4,max,2.9999828E7,98.73,101.0,12500.0,125.0,625.0,2750.0,1375.0,10875.0,57.0,91.0,100.0,100.0,100.0,9.1,91.0,50.0,262.0,61.0,155.0,38.0,3565.0,184.0,26.5,27.4,150.0,193.3,248.0,23.9,86.0,70.4,142.0,2947.0,9.0,161.0,12500.0,38.0,1.0,1.32,38.0,61854.0,3756.0,42606.0,82.2,20.0,M


we could see there are unusually high  values by looking at the max values for each columns. Hence we will go for removing the outliers.

In [8]:
import pyspark.pandas as ps


C:\Spark\spark-3.3.2-bin-hadoop2\python\lib\pyspark.zip\pyspark\pandas\__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.


In [10]:
cols = model_data.columns
#cols.remove("subject_id")
cols.remove("hadm_id")
print(cols)
df = ps.DataFrame(model_data)

['age', 'bg2_lactate', 'db_wbc', 'basophils_abs', 'eosinophils_abs', 'lymphocytes_abs', 'monocytes_abs', 'neutrophils_abs', 'basophils', 'eosinophils', 'lymphocytes', 'monocytes', 'neutrophils', 'albumin', 'aniongap', 'bicarbonate', 'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium', 'potassium', 'inr', 'pt', 'ptt', 'hematocrit', 'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet', 'rbc', 'rdw', 'wbc', 'scr_min', 'ckd', 'mdrd_est', 'scr_baseline', 'alt', 'alp', 'ast', 'bilirubin_total', 'charlson_comorbidity_index', 'gender']


In [11]:
model_data.printSchema()

root
 |-- hadm_id: double (nullable = true)
 |-- age: double (nullable = true)
 |-- bg2_lactate: double (nullable = true)
 |-- db_wbc: double (nullable = true)
 |-- basophils_abs: double (nullable = true)
 |-- eosinophils_abs: double (nullable = true)
 |-- lymphocytes_abs: double (nullable = true)
 |-- monocytes_abs: double (nullable = true)
 |-- neutrophils_abs: double (nullable = true)
 |-- basophils: double (nullable = true)
 |-- eosinophils: double (nullable = true)
 |-- lymphocytes: double (nullable = true)
 |-- monocytes: double (nullable = true)
 |-- neutrophils: double (nullable = true)
 |-- albumin: double (nullable = true)
 |-- aniongap: double (nullable = true)
 |-- bicarbonate: double (nullable = true)
 |-- bun: double (nullable = true)
 |-- calcium: double (nullable = true)
 |-- chloride: double (nullable = true)
 |-- creatinine: double (nullable = true)
 |-- glucose: double (nullable = true)
 |-- sodium: double (nullable = true)
 |-- potassium: double (nullable = true)
 |

In [16]:
# obtain the upper and lower fence values to remove outliers
cols = model_data.columns
cols.remove("gender")
cols.remove("hadm_id")
limit_dict = {}
print(cols)
for c in cols:
    print(c)
    fig = df[c].plot.box()
    limit_dict[c + "_upper"] = fig["data"][0]["upperfence"][0]
    limit_dict[c + "_lower"] = fig["data"][0]["lowerfence"][0]
    print(fig)
    fig.show()
print(limit_dict)

['hadm_id', 'age', 'bg2_lactate', 'db_wbc', 'basophils_abs', 'eosinophils_abs', 'lymphocytes_abs', 'monocytes_abs', 'neutrophils_abs', 'basophils', 'eosinophils', 'lymphocytes', 'monocytes', 'neutrophils', 'albumin', 'aniongap', 'bicarbonate', 'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium', 'potassium', 'inr', 'pt', 'ptt', 'hematocrit', 'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet', 'rbc', 'rdw', 'wbc', 'scr_min', 'ckd', 'mdrd_est', 'scr_baseline', 'alt', 'alp', 'ast', 'bilirubin_total', 'charlson_comorbidity_index']
hadm_id
Figure({
    'data': [{'boxpoints': 'suspectedoutliers',
              'lowerfence': [20000019.0],
              'mean': [24993919.76303571],
              'median': [24952027.0],
              'name': 'hadm_id',
              'notched': False,
              'q1': [22484325.0],
              'q3': [27417588.0],
              'type': 'box',
              'upperfence': [29999828.0]}],
    'layout': {'template': '...', 'xaxis': {'title': {'text': 'ha

age
Figure({
    'data': [{'boxpoints': 'suspectedoutliers',
              'lowerfence': [18.0],
              'mean': [60.41605681473008],
              'median': [61.9],
              'name': 'age',
              'notched': False,
              'q1': [47.65],
              'q3': [74.67],
              'type': 'box',
              'upperfence': [98.73]}],
    'layout': {'template': '...', 'xaxis': {'title': {'text': 'age'}}, 'yaxis': {'title': {'text': 'value'}}}
})


bg2_lactate
Figure({
    'data': [{'boxpoints': 'suspectedoutliers',
              'lowerfence': [0.2],
              'mean': [2.4679610986719545],
              'median': [1.8],
              'name': 'bg2_lactate',
              'notched': False,
              'q1': [1.3],
              'q3': [2.7],
              'type': 'box',
              'upperfence': [4.8],
              'y': [[4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9,
                    4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9,
                    4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9,
                    4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9,
                    4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9,
                    4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9,
                    4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9,
                    4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9

db_wbc
Figure({
    'data': [{'boxpoints': 'suspectedoutliers',
              'lowerfence': [0.2],
              'mean': [11.623239965285308],
              'median': [9.7],
              'name': 'db_wbc',
              'notched': False,
              'q1': [7.2],
              'q3': [13.5],
              'type': 'box',
              'upperfence': [22.9],
              'y': [[23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0,
                    23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0,
                    23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0,
                    23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0,
                    23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0,
                    23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0,
                    23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0,
                    23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0,
 

basophils_abs
Figure({
    'data': [{'boxpoints': 'suspectedoutliers',
              'lowerfence': [0.0],
              'mean': [0.061629758214501465],
              'median': [0.04],
              'name': 'basophils_abs',
              'notched': False,
              'q1': [0.02],
              'q3': [0.06],
              'type': 'box',
              'upperfence': [0.12],
              'y': [[0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13,
                    0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13,
                    0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13,
                    0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13,
                    0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13,
                    0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13,
                    0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13,
                    0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0

...

alp
Figure({
    'data': [{'boxpoints': 'suspectedoutliers',
              'lowerfence': [6.0],
              'mean': [121.09907908502642],
              'median': [82.0],
              'name': 'alp',
              'notched': False,
              'q1': [63.0],
              'q3': [119.0],
              'type': 'box',
              'upperfence': [203.0],
              'y': [[204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0,
                    204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0,
                    204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0,
                    204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0,
                    204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0,
                    204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0,
                    204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0, 204.0,
                    204.0, 204.0, 205.0, 205.0, 205.0, 205.0, 205.0, 

ast
Figure({
    'data': [{'boxpoints': 'suspectedoutliers',
              'lowerfence': [1.0],
              'mean': [148.9965000063177],
              'median': [30.0],
              'name': 'ast',
              'notched': False,
              'q1': [20.0],
              'q3': [63.0],
              'type': 'box',
              'upperfence': [127.0],
              'y': [[128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0,
                    128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0,
                    128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0,
                    128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0,
                    128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0,
                    128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0,
                    128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0,
                    128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 12

bilirubin_total
Figure({
    'data': [{'boxpoints': 'suspectedoutliers',
              'lowerfence': [0.1],
              'mean': [1.4387537698849087],
              'median': [0.6],
              'name': 'bilirubin_total',
              'notched': False,
              'q1': [0.4],
              'q3': [1.0],
              'type': 'box',
              'upperfence': [1.9],
              'y': [[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,
                    2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,
                    2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,
                    2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,
                    2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,
                    2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,
                    2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,
                    2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 

charlson_comorbidity_index
Figure({
    'data': [{'boxpoints': 'suspectedoutliers',
              'lowerfence': [0.0],
              'mean': [3.809985969885879],
              'median': [3.0],
              'name': 'charlson_comorbidity_index',
              'notched': False,
              'q1': [1.0],
              'q3': [6.0],
              'type': 'box',
              'upperfence': [13.0],
              'y': [[14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0,
                    14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0,
                    14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0,
                    14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0,
                    14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0,
                    14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0,
                    14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0,
                    14.0, 14.0, 14.0, 14.0,

{'hadm_id_upper': 29999828.0, 'hadm_id_lower': 20000019.0, 'age_upper': 98.73, 'age_lower': 18.0, 'bg2_lactate_upper': 4.8, 'bg2_lactate_lower': 0.2, 'db_wbc_upper': 22.9, 'db_wbc_lower': 0.2, 'basophils_abs_upper': 0.12, 'basophils_abs_lower': 0.0, 'eosinophils_abs_upper': 0.57, 'eosinophils_abs_lower': 0.0, 'lymphocytes_abs_upper': 4.42, 'lymphocytes_abs_lower': 0.0, 'monocytes_abs_upper': 1.63, 'monocytes_abs_lower': 0.0, 'neutrophils_abs_upper': 21.74, 'neutrophils_abs_lower': 0.0, 'basophils_upper': 1.2, 'basophils_lower': 0.0, 'eosinophils_upper': 6.5, 'eosinophils_lower': 0.0, 'lymphocytes_upper': 50.8, 'lymphocytes_lower': 0.0, 'monocytes_upper': 14.6, 'monocytes_lower': 0.0, 'neutrophils_upper': 100.0, 'neutrophils_lower': 35.7, 'albumin_upper': 5.3, 'albumin_lower': 1.8, 'aniongap_upper': 23.0, 'aniongap_lower': 7.0, 'bicarbonate_upper': 35.0, 'bicarbonate_lower': 19.0, 'bun_upper': 44.0, 'bun_lower': 1.0, 'calcium_upper': 10.6, 'calcium_lower': 7.4, 'chloride_upper': 117.0, 

#### Remove all data above upper fence and below lower fence

In [31]:
model_data = model_data.withColumn('outlier', F.lit(0))
for c in model_data.columns:
    if (c not in ("subject_id", "gender", "hadm_id", "outlier")):
        model_data = model_data.withColumn('outlier', F.when(F.col(c) > limit_dict[c + "_upper"], 1) \
                                                          .when(F.col(c) < limit_dict[c + "_lower"], 1) \
                                                        .otherwise(F.col("outlier")))

In [32]:
model_data.printSchema()

root
 |-- hadm_id: double (nullable = true)
 |-- age: double (nullable = true)
 |-- bg2_lactate: double (nullable = true)
 |-- db_wbc: double (nullable = true)
 |-- basophils_abs: double (nullable = true)
 |-- eosinophils_abs: double (nullable = true)
 |-- lymphocytes_abs: double (nullable = true)
 |-- monocytes_abs: double (nullable = true)
 |-- neutrophils_abs: double (nullable = true)
 |-- basophils: double (nullable = true)
 |-- eosinophils: double (nullable = true)
 |-- lymphocytes: double (nullable = true)
 |-- monocytes: double (nullable = true)
 |-- neutrophils: double (nullable = true)
 |-- albumin: double (nullable = true)
 |-- aniongap: double (nullable = true)
 |-- bicarbonate: double (nullable = true)
 |-- bun: double (nullable = true)
 |-- calcium: double (nullable = true)
 |-- chloride: double (nullable = true)
 |-- creatinine: double (nullable = true)
 |-- glucose: double (nullable = true)
 |-- sodium: double (nullable = true)
 |-- potassium: double (nullable = true)
 |

In [33]:
model_data.groupBy("outlier").count().toPandas()

,outlier,count
0,1,74387
1,0,63887


In [34]:
model_data_filtered = model_data.filter("outlier = 0")
model_data_filtered.count()

63887

Filtered out 63K patient data for modelling. Note: Outlier detection can be further tuned to adjust the upper and lower range.

In [35]:
model_data_filtered.describe().toPandas()

,summary,hadm_id,age,bg2_lactate,db_wbc,basophils_abs,eosinophils_abs,lymphocytes_abs,monocytes_abs,neutrophils_abs,basophils,eosinophils,lymphocytes,monocytes,neutrophils,albumin,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,inr,pt,ptt,hematocrit,hemoglobin,mch,mchc,mcv,platelet,rbc,rdw,wbc,scr_min,ckd,mdrd_est,scr_baseline,alt,alp,ast,bilirubin_total,charlson_comorbidity_index,gender,outlier
0,count,63887,63887,17273,63885,29294,29298,29323,29294,29294,29294,29294,29323,29294,29294,18614,63019,63023,63114,55228,63041,63189,63007,63047,63052,38208,38203,37204,63887,63885,63884,63884,63884,63874,63884,63882,63885,63189,63887,63887,63887,25638,24623,25597,24369,63887,63887,63887
1,mean,2.5000434834629893E7,56.82400472709627,1.8289445956116495,9.807902637551857,0.03798183928449568,0.13908423783193294,1.8242355829894628,0.5756762476957737,7.385843858810682,0.42368403085956235,1.6212330170000704,20.932731985131134,6.198129309756263,72.01603400013661,3.8106855055334714,14.327044224757612,26.91277787474414,15.677789396964224,8.971773375823851,104.77436906140449,0.8812862998306659,123.15828400019045,140.30542293844275,4.211982173444147,1.1520927554438962,12.809534068005142,30.201865390818195,37.577690296930484,12.582841042498234,30.448135683426205,33.90112391209065,90.68908333855113,245.37677302188683,4.175787208064622,13.625243417551095,9.807835329106993,0.8812862998306659,0.0,0.9607652574076345,0.855899322240839,26.779780014041656,76.52617471469765,29.306910966128843,0.5620665599737384,2.7256405841564013,None,0.0
2,stddev,2890324.5420365734,19.28094185711829,0.8446524036275503,3.8157237124384538,0.0233368994131413,0.11716537032115232,0.832592610036045,0.28397005240769657,3.7497752623962906,0.26361586961431727,1.3973167914971472,10.602560077346135,2.657450674718072,12.371293813555372,0.5720379854299908,2.6385611231608976,2.867133655093651,6.679302604463797,0.5364111589938664,3.5823892112400713,0.2397615797856708,34.283543246721926,2.7553455899069355,0.4200204113241376,0.16100944616222626,1.7375978469857747,5.576755160204632,4.668777216754638,1.6635774665861176,1.8780429152082332,1.2829131012811736,5.10701980108691,74.10389589735497,0.561571616438411,1.1028460946027494,3.8157183336050604,0.2397615797856708,0.0,0.14264437270439068,0.17272703592571728,18.968312549900876,28.96570064474761,18.355941082843085,0.33419516072408995,2.532551218466355,None,0.0
3,min,2.0000019E7,18.0,0.2,0.9,0.0,0.0,0.0,0.0,0.47,0.0,0.0,0.0,0.0,35.8,1.8,7.0,19.0,1.0,7.4,93.0,0.2,31.0,133.0,3.0,0.8,8.5,17.3,22.7,7.3,25.1,30.1,75.0,17.0,2.4,10.6,0.9,0.2,0.0,0.76,0.4,1.0,6.0,2.0,0.1,0.0,F,0
4,max,2.9999723E7,98.51,4.8,22.9,0.12,0.57,4.42,1.63,21.72,1.2,6.5,50.8,14.6,100.0,5.3,23.0,35.0,44.0,10.6,117.0,1.9,250.0,149.0,5.7,1.8,21.0,57.4,51.0,17.2,35.8,37.6,107.0,484.0,5.79,18.1,22.9,1.9,0.0,1.32,1.32,111.0,203.0,127.0,1.9,13.0,M,0


In [36]:
model_data.filter("db_wbc = 12500").toPandas()

,hadm_id,age,bg2_lactate,db_wbc,basophils_abs,eosinophils_abs,lymphocytes_abs,monocytes_abs,neutrophils_abs,basophils,eosinophils,lymphocytes,monocytes,neutrophils,albumin,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,inr,pt,ptt,hematocrit,hemoglobin,mch,mchc,mcv,platelet,rbc,rdw,wbc,scr_min,ckd,mdrd_est,scr_baseline,alt,alp,ast,bilirubin_total,charlson_comorbidity_index,gender,outlier
0,23028159.0,46.14,7.7,12500.0,125.0,625.0,2750.0,1375.0,10875.0,1.0,5.0,22.0,11.0,87.0,2.9,22.0,32.0,29.0,10.7,116.0,2.1,1180.0,147.0,6.2,2.2,23.2,102.7,38.7,13.1,29.9,36.1,96.0,592.0,9.0,19.1,12500.0,2.1,0.0,0.86,0.86,148.0,505.0,185.0,8.3,5.0,F,1


In [37]:
model_data_filtered.groupBy("charlson_comorbidity_index").count().toPandas()

,charlson_comorbidity_index,count
0,8.0,1319
1,0.0,15677
2,7.0,1978
3,1.0,9489
4,4.0,7732
5,11.0,327
6,3.0,8010
7,2.0,8874
8,10.0,593
9,13.0,81


We have a reasonable distribution of patient data with different comorbidity index.

In [38]:
model_data_filtered.columns

['hadm_id',
 'age',
 'bg2_lactate',
 'db_wbc',
 'basophils_abs',
 'eosinophils_abs',
 'lymphocytes_abs',
 'monocytes_abs',
 'neutrophils_abs',
 'basophils',
 'eosinophils',
 'lymphocytes',
 'monocytes',
 'neutrophils',
 'albumin',
 'aniongap',
 'bicarbonate',
 'bun',
 'calcium',
 'chloride',
 'creatinine',
 'glucose',
 'sodium',
 'potassium',
 'inr',
 'pt',
 'ptt',
 'hematocrit',
 'hemoglobin',
 'mch',
 'mchc',
 'mcv',
 'platelet',
 'rbc',
 'rdw',
 'wbc',
 'scr_min',
 'ckd',
 'mdrd_est',
 'scr_baseline',
 'alt',
 'alp',
 'ast',
 'bilirubin_total',
 'charlson_comorbidity_index',
 'gender',
 'outlier']

#### Removed unnecessary columns and duplicate columns

In [41]:
model_data_filtered = model_data_filtered.drop("hadm_id", "db_wbc", "basophils_abs", "eosinophils_abs", "lymphocytes_abs", "monocytes_abs", "neutrophils_abs", "outlier")

In [42]:
len(model_data_filtered.columns)

39

After removing unnecessary columns and duplicates, we have 39 columns left for model building.

### Handling missing values

In [44]:
model_data_filtered.columns

['age',
 'bg2_lactate',
 'basophils',
 'eosinophils',
 'lymphocytes',
 'monocytes',
 'neutrophils',
 'albumin',
 'aniongap',
 'bicarbonate',
 'bun',
 'calcium',
 'chloride',
 'creatinine',
 'glucose',
 'sodium',
 'potassium',
 'inr',
 'pt',
 'ptt',
 'hematocrit',
 'hemoglobin',
 'mch',
 'mchc',
 'mcv',
 'platelet',
 'rbc',
 'rdw',
 'wbc',
 'scr_min',
 'ckd',
 'mdrd_est',
 'scr_baseline',
 'alt',
 'alp',
 'ast',
 'bilirubin_total',
 'charlson_comorbidity_index',
 'gender']

In [45]:
model_data_filtered.describe().toPandas()

,summary,age,bg2_lactate,basophils,eosinophils,lymphocytes,monocytes,neutrophils,albumin,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,inr,pt,ptt,hematocrit,hemoglobin,mch,mchc,mcv,platelet,rbc,rdw,wbc,scr_min,ckd,mdrd_est,scr_baseline,alt,alp,ast,bilirubin_total,charlson_comorbidity_index,gender
0,count,63887,17273,29294,29294,29323,29294,29294,18614,63019,63023,63114,55228,63041,63189,63007,63047,63052,38208,38203,37204,63887,63885,63884,63884,63884,63874,63884,63882,63885,63189,63887,63887,63887,25638,24623,25597,24369,63887,63887
1,mean,56.82400472709627,1.8289445956116495,0.42368403085956235,1.6212330170000704,20.932731985131134,6.198129309756263,72.01603400013661,3.8106855055334714,14.327044224757612,26.91277787474414,15.677789396964224,8.971773375823851,104.77436906140449,0.8812862998306659,123.15828400019045,140.30542293844275,4.211982173444147,1.1520927554438962,12.809534068005142,30.201865390818195,37.577690296930484,12.582841042498234,30.448135683426205,33.90112391209065,90.68908333855113,245.37677302188683,4.175787208064622,13.625243417551095,9.807835329106993,0.8812862998306659,0.0,0.9607652574076345,0.855899322240839,26.779780014041656,76.52617471469765,29.306910966128843,0.5620665599737384,2.7256405841564013,None
2,stddev,19.28094185711829,0.8446524036275503,0.26361586961431727,1.3973167914971472,10.602560077346135,2.657450674718072,12.371293813555372,0.5720379854299908,2.6385611231608976,2.867133655093651,6.679302604463797,0.5364111589938664,3.5823892112400713,0.2397615797856708,34.283543246721926,2.7553455899069355,0.4200204113241376,0.16100944616222626,1.7375978469857747,5.576755160204632,4.668777216754638,1.6635774665861176,1.8780429152082332,1.2829131012811736,5.10701980108691,74.10389589735497,0.561571616438411,1.1028460946027494,3.8157183336050604,0.2397615797856708,0.0,0.14264437270439068,0.17272703592571728,18.968312549900876,28.96570064474761,18.355941082843085,0.33419516072408995,2.532551218466355,None
3,min,18.0,0.2,0.0,0.0,0.0,0.0,35.8,1.8,7.0,19.0,1.0,7.4,93.0,0.2,31.0,133.0,3.0,0.8,8.5,17.3,22.7,7.3,25.1,30.1,75.0,17.0,2.4,10.6,0.9,0.2,0.0,0.76,0.4,1.0,6.0,2.0,0.1,0.0,F
4,max,98.51,4.8,1.2,6.5,50.8,14.6,100.0,5.3,23.0,35.0,44.0,10.6,117.0,1.9,250.0,149.0,5.7,1.8,21.0,57.4,51.0,17.2,35.8,37.6,107.0,484.0,5.79,18.1,22.9,1.9,0.0,1.32,1.32,111.0,203.0,127.0,1.9,13.0,M


In [47]:
# impute with median values for remaining missing values
from pyspark.ml.feature import Imputer
impute_numeric_cols = model_data_filtered.columns
impute_numeric_cols.remove("gender")
imputer = Imputer(
    inputCols = impute_numeric_cols, 
    outputCols = ["{}_imputed".format(c) for c in impute_numeric_cols]
    ).setStrategy("median")
model = imputer.fit(model_data_filtered)
model_data_transformed = model.transform(model_data_filtered)

In [48]:
model_data_transformed.describe().toPandas()

,summary,age,bg2_lactate,basophils,eosinophils,lymphocytes,monocytes,neutrophils,albumin,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,inr,pt,ptt,hematocrit,hemoglobin,mch,mchc,mcv,platelet,rbc,rdw,wbc,scr_min,ckd,mdrd_est,scr_baseline,alt,alp,ast,bilirubin_total,charlson_comorbidity_index,gender,age_imputed,bg2_lactate_imputed,basophils_imputed,eosinophils_imputed,lymphocytes_imputed,monocytes_imputed,neutrophils_imputed,albumin_imputed,aniongap_imputed,bicarbonate_imputed,bun_imputed,calcium_imputed,chloride_imputed,creatinine_imputed,glucose_imputed,sodium_imputed,potassium_imputed,inr_imputed,pt_imputed,ptt_imputed,hematocrit_imputed,hemoglobin_imputed,mch_imputed,mchc_imputed,mcv_imputed,platelet_imputed,rbc_imputed,rdw_imputed,wbc_imputed,scr_min_imputed,ckd_imputed,mdrd_est_imputed,scr_baseline_imputed,alt_imputed,alp_imputed,ast_imputed,bilirubin_total_imputed,charlson_comorbidity_index_imputed
0,count,63887,17273,29294,29294,29323,29294,29294,18614,63019,63023,63114,55228,63041,63189,63007,63047,63052,38208,38203,37204,63887,63885,63884,63884,63884,63874,63884,63882,63885,63189,63887,63887,63887,25638,24623,25597,24369,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887,63887
1,mean,56.82400472709627,1.8289445956116495,0.42368403085956235,1.6212330170000704,20.932731985131134,6.198129309756263,72.01603400013661,3.8106855055334714,14.327044224757612,26.91277787474414,15.677789396964224,8.971773375823851,104.77436906140449,0.8812862998306659,123.15828400019045,140.30542293844275,4.211982173444147,1.1520927554438962,12.809534068005142,30.201865390818195,37.577690296930484,12.582841042498234,30.448135683426205,33.90112391209065,90.68908333855113,245.37677302188683,4.175787208064622,13.625243417551095,9.807835329106993,0.8812862998306659,0.0,0.9607652574076345,0.855899322240839,26.779780014041656,76.52617471469765,29.306910966128843,0.5620665599737384,2.7256405841564013,None,56.82400472709627,1.6618992909357377,0.41085979933323036,1.4472944417486133,20.26580211936804,6.036700737239697,72.54882370435293,3.8739774915089122,14.322600842111854,26.913957456133485,15.669588492181507,8.975599104669177,104.77735689576909,0.8803982030773088,123.04590918340195,140.30140717203813,4.21182556701677,1.1311543819557521,12.685094463662413,29.78342698827827,37.577690296930484,12.58284157966409,30.44813811886612,33.90112385931408,90.68909793854776,245.37486499600857,4.175787405888526,13.62522578928419,9.807813170128513,0.8803982030773088,0.0,0.9607652574076345,0.855899322240839,22.720741308873478,73.12986992658914,25.52693036141938,0.5236746129885593,2.7256405841564013
2,stddev,19.28094185711829,0.8446524036275503,0.26361586961431727,1.3973167914971472,10.602560077346135,2.657450674718072,12.371293813555372,0.5720379854299908,2.6385611231608976,2.867133655093651,6.679302604463797,0.5364111589938664,3.5823892112400713,0.2397615797856708,34.283543246721926,2.7553455899069355,0.4200204113241376,0.16100944616222626,1.7375978469857747,5.576755160204632,4.668777216754638,1.6635774665861176,1.8780429152082332,1.2829131012811736,5.10701980108691,74.10389589735497,0.561571616438411,1.1028460946027494,3.8157183336050604,0.2397615797856708,0.0,0.14264437270439068,0.17272703592571728,18.968312549900876,28.96570064474761,18.355941082843085,0.33419516072408995,2.532551218466355,None,19.28094185711829,0.45080230092950563,0.17889477756433583,0.9596238678234632,7.20920004329376,1.8055899982717187,8.391445135120632,0.3114222348201369,2.6208486378117435,2.8476977920931095,6.6391844888635605,0.4988295388939295,3.558684024088815,0.23859787593365497,34.059880392372044,2.7373925256555105,0.4172687362614888,0.12710702741208696,1.3522042469266164,4.284254865615289,4.668777216754638,1.6635514293713365,1.8779988531436762,1.2828829790201604,5.106900334736289

In [51]:
print(len(model_data_transformed.columns))
condition = lambda x: ("_imputed" in x)
model_data_final = model_data_transformed.select(*filter(condition, model_data_transformed.columns), "gender")
print(len(model_data_final.columns))
model_data_final.columns

77
39


['age_imputed',
 'bg2_lactate_imputed',
 'basophils_imputed',
 'eosinophils_imputed',
 'lymphocytes_imputed',
 'monocytes_imputed',
 'neutrophils_imputed',
 'albumin_imputed',
 'aniongap_imputed',
 'bicarbonate_imputed',
 'bun_imputed',
 'calcium_imputed',
 'chloride_imputed',
 'creatinine_imputed',
 'glucose_imputed',
 'sodium_imputed',
 'potassium_imputed',
 'inr_imputed',
 'pt_imputed',
 'ptt_imputed',
 'hematocrit_imputed',
 'hemoglobin_imputed',
 'mch_imputed',
 'mchc_imputed',
 'mcv_imputed',
 'platelet_imputed',
 'rbc_imputed',
 'rdw_imputed',
 'wbc_imputed',
 'scr_min_imputed',
 'ckd_imputed',
 'mdrd_est_imputed',
 'scr_baseline_imputed',
 'alt_imputed',
 'alp_imputed',
 'ast_imputed',
 'bilirubin_total_imputed',
 'charlson_comorbidity_index_imputed',
 'gender']

In [52]:
model_data_final.toPandas().to_csv("../data/EDA/final_data_for_modelling.csv")